
#1.0 - Tabela Utilizada no Drive

Estou utilizando a ultima tabela disponibilizada pela MEDICINA USP.<br>
Nome da Tabela: NEW-BDIPMama-INTELI(1).csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1.1 - Buscando a tabela e importando as bibliotecas utilizadas
Nomeei a tabela como tabela_completa

In [ ]:
import pandas as pd
tabela_completa = pd.read_csv ('/content/drive/MyDrive/DRIVE - USPMedicina/NEW-BDIPMamaV11-INTELI (3).csv',delimiter=";")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22,24,25,75,76,77,78,89,90,91,92) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
tabela_completa['redcap_repeat_instrument'].value_counts()

# 2.0 - Observando os dados válidos
Para cada coluna da tabela, consultamos com o value_counts(), afim de consultar a quantidade de dados nas colunas, sendo assim, há a consulta para uma primeira seleção da tabela.<br><br>
Importante destacar que todas as colunas selecionadas até o momento tem mais de 1/3 dos registros válidos.
O restante foi ignorado devido à ausência de registros significativos.

# 3.0 - Premissas assumidas

Assumimos que todos os NaN acima do primeiro dado numérico significa que o dado não foi preenchido no cadastro do paciente, sendo assim, transformaremos este dado nulo em 0.0<br>
Para não perder todos os dados da amostra foi preferível substituir os nulos (Nan) pelo número 0.0 até chegar o primeiro exposto na coluna, este foi replicado na coluna abaixo até encontrar o próximo valor válido, segundo orientações docente para o entendimento dos dados. A partir deste momento, lê-se a tabela da seguinte forma:<br>

*   Caso observe uma coluna onde o primeiro dado é -1, significa que este é Nan.

*   Caso encontre uma coluna onde o primeiro dado é 0, este dado é lido como Não.

Optamos por renomear a tabela completa, que está no drive, para tabela tratada, a fim de conseguir fazer consultas na tabela completa ao longo do projeto, sem prejudicar a modelagem dos dados que ocorre simultaneamente.<br><br>

Utilizamos o comando fillna para buscar na coluna um valor válido e repetir nos dados vazios que iriam se repetir normalmente para o mesmo record_id. Posteriormente,a fim de transformar as string dessa coluna em novas colunas com dados numéricos, especificamente, dados de afirmação (Sim) ou negação (Não), utilizamos o método dummies.

Sendo assim, lê-se a tabela da seguinte forma:

*   Caso observe uma coluna onde o primeiro dado é -1, significa que este é **Nan**;
*   Caso observe uma coluna onde o primeiro dado é 0, significa que este é lido como **Não**;
*   Caso Observe uma coluna onde o primeiro dado é 1, significa que este é lido como **Sim**;

Fizemos este comando para a coluna "redcap_repeat_instrument" visando replicar a coluna em três dados diferentes, pegando seus respectivos valores e transformando em colunas.

In [ ]:
tabela_trat = tabela_completa.fillna(method='ffill').dropna(subset=['redcap_repeat_instrument']) #Utilizei o fillna com o método ffill para buscar o último valor válido e repetir na linha posterior.
tabela_trat['redcap_repeat_instrument'].unique()

tabela_tratada = pd.get_dummies(tabela_trat, columns=['redcap_repeat_instrument', 'antec_fam_cancer_mama', 'ultinfo']) #Utilizei o método get_dummies na coluna redcap_repeat_instrument para separar o conteúdo dela em novas colunas no final

A saída esperada é a exclusão da coluna "redcap_repeat_instrument", e a criação de três novas colunas, respectivamente:
* "redcap_repeat_instrument_dados_antropometricos";
* "redcap_repeat_instrument_dados_histopatologicos_mama"; e 
* "redcap_repeat_instrument_registro_de_tumores".

Sendo assim, indicando 1 e 0 nos registros do paciente.<br>

Fiz o mesmo processo para a coluna "antec_fam_cancer_mama" que indica se há algum familiar com câncer ou não, buscando transformar as string "Sim" e "Não" em 0 e 1.

Selecionamos as colunas que iremos utilizar, como retiramos as duplicatas com base 

In [ ]:
ta_final = tabela_tratada[['record_id','t','n','m', 'ultinfo_1.0',	'ultinfo_2.0',	'ultinfo_3.0',	'ultinfo_4.0','breast_feeding_time', 'dob','tumor_subtype',
                           'antec_fam_cancer_mama_Não','antec_fam_cancer_mama_Sim', 'tempo_rep_hormo','bmi', 'follow_up_days', 'menarche', 'tobaco','alcohol',
                           'date_last_fu','familial_degree___1','familial_degree___2','familial_degree___3', 'primary_diganosis','benign']]
tabela_final = ta_final.fillna(-1)
tabela_final = tabela_final.drop_duplicates(subset='record_id', keep='last', ignore_index=True)

# 4.0 - Transformando Colunas
Foi necessário transformar a coluna data de nascimento (dob) em idade, visto que será mais fácil saber a idade da paciente quando gerarmos o modelo de predição.<br><br>
A data de nascimento foi calculada até a última consulta do mesmo, e não até a data presente. Preferimos demonstrar desta forma devido a não existência da data de óbito no banco de dados, e também, por nos concentrarmos em encontrar a relação da sobrevida do paciente até a ultima consulta.
<br><br>
Abaixo o código que gera a idade a partir da data de nascimento.

In [ ]:
from datetime import datetime, date
tabela_final['dob']
def pegar_idade(nascimento, _date):
  nascimento = datetime.strptime(nascimento,"%d/%m/%Y").date()
  _date = datetime.strptime(_date,"%d/%m/%Y").date()
  idade = _date.year - nascimento.year
  if(nascimento.month == _date.month and nascimento.day >= _date.year):
    idade = idade + 1
  elif(nascimento.month > _date.month):
    idade = idade + 1
  return idade

def convertDate(age):
  return datetime.strptime(age,"%d/%m/%Y").date()
tabela_final['dob'].apply(convertDate)
tabela_final['date_last_fu'].apply(convertDate)
tabela_final['idade'] = ""
for ind in tabela_final.index:
    tabela_final['idade'][ind] = pegar_idade(tabela_final['dob'][ind], tabela_final['date_last_fu'][ind])



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Com o intuito de realizar testes por meio de modelos de classificação, foi necessário transformar a coluna do dias de sobrevida em um filtro, de 1 a 5, para que os dados pudessem ser melhor enquadrados dentro da aplicação do modelo e evitasse ruídos nos resultados. A seguir, segue o código da função responsável por essa conversão:

In [ ]:
def getCategory(x):
   if x<=900: return 1;
   elif x>=901 and x<=1800: return 2 
   elif  x>=1801 and x<=2700: return 3
   elif x>=2701 and x<=3600: return 4
   else: return 5

tabela_final['sobrevida'] = tabela_final['follow_up_days'].apply(lambda x: getCategory(x))
tabela_final['sobrevida'].value_counts()

2    2197
1     891
4     529
3     461
5      56
Name: sobrevida, dtype: int64

Com a finalidade de facilitar a visualização e compreensão das colunas utilizadas no modelo, foi necessário renomear algumas Features, que podem ser visualizadas no código abaixo:

In [ ]:
tabela_final.rename({'ultinfo_1.0':'ultinfo_1','ultinfo_2.0':'ultinfo_2',
                     'ultinfo_3.0':'ultinfo_3', 'ultinfo_4.0':'ultinfo_4',
                     'record_id':'pacientes', 't':'tumor', 'n':'nodulo',
                     'm':'metastese', 'breast_feeding_time':'tempo_de_amamentacao',
                     'dob':'data_de_nascimento', 'tobaco':'fumante', 'alcohol':'alcolatra',
                     'date_last_fu':'data_ultima_consulta', 'familial_degree___1':'grau_parentesco_1',
                     'familial_degree___2':'grau_parentesco_2', 'familial_degree___3':'grau_parentesco_3',
                     'primary_diganosis':'tipo_tumor', 'tumor_subtype':'subtipo_tumoral',
                     'menarche':'primeira_menstruacao', 'bmi':'IMC', 
                     'benign':'benigno/maligno'}, axis='columns', inplace=True)

Foi utilizado o método query para pegar apenas a ultima informação dos pacientes que já foram a óbito por Câncer, para que seja possível conferir com o mínimo de viés possível os dados, pois partimos do suposto que os dados dos que morreram com Câncer 

In [ ]:
x_inter3 = tabela_final.query('ultinfo_3 == 1')

In [ ]:
x_inter3

,pacientes,tumor,nodulo,metastese,ultinfo_1,ultinfo_2,ultinfo_3,ultinfo_4,tempo_de_amamentacao,data_de_nascimento,...,fumante,alcolatra,data_ultima_consulta,grau_parentesco_1,grau_parentesco_2,grau_parentesco_3,tipo_tumor,benigno/maligno,idade,sobrevida
5,1705,1.0,1.0,0.0,0,0,1,0,-1.0,09/11/1963,...,2.0,2.0,11/06/2013,0.0,0.0,0.0,2.0,2.0,51,3
8,1898,4.0,2.0,1.0,0,0,1,0,-1.0,13/10/1947,...,2.0,2.0,22/08/2009,0.0,0.0,0.0,2.0,2.0,63,1
12,2016,3.0,3.0,0.0,0,0,1,0,-1.0,17/01/1966,...,2.0,2.0,22/06/2011,0.0,0.0,0.0,2.0,2.0,45,2
19,2350,3.0,2.0,0.0,0,0,1,0,-1.0,23/03/1936,...,2.0,2.0,22/07/2010,0.0,0.0,0.0,2.0,2.0,74,2
20,2370,2.0,1.0,0.0,0,0,1,0,-1.0,18/05/1958,...,2.0,2.0,03/11/2009,0.0,0.0,0.0,2.0,2.0,51,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4054,79944,42.0,1.0,0.0,0,0,1,0,12.0,04/01/1984,...,3.0,3.0,05/05/2021,1.0,1.0,0.0,2.0,2.0,37,1
4069,80274,3.0,1.0,0.0,0,0,1,0,24.0,12/05/1985,...,2.0,2.0,06/08/2019,0.0,0.0,0.0,2.0,2.0,34,1
4080,80930,3.0,1.0,0.0,0,0,1,0,60.0,11/06/1959,...,2.0,2.0,06/08/2021,0.0,0.0,0.0,2.0,2.0,62,1
4111,81873,42.0,1.0,1.0,0,0,1,0,6.0,11/06/1959,...,2.0,2.0,12/05/2020,0.0,0.0,0.0,2.0,2.0,62,1


#5.0 - Testando o Modelo Preditivo#

Importando biblioteca responsável pela utilização de métricas de desempenho dos modelos de aprendizagem de máquina

In [ ]:
from scipy.sparse.linalg.isolve.minres import sqrt
acuraciaModelos = {}

def calc_MCC(res):
  TP = res[0][0]
  TN = res[1][1]
  FP = res[0][1]
  FN = res[1][0]
  raiz = (TP+FP) * (TP+FN) * (TN+FP) * (TN+FN)
  MCC = ((TP*TN)-(FP*FN))/sqrt(raiz)
  return MCC

##**5.1 Regressão Linear**
 Regressão linear é um conjunto de técnicas que se baseiam na ideia de estimar o valor de uma variável Y através dos valores de X, permitindo assim que um conjunto de dados sirvam de parâmetro para a estimativa de um valor desejado. Após serem submetidos ao processo de normalização padrão, que se baseia em limitar os dados entre 0 e 1 para deixá-los dimensionados da mesma forma, o que resulta em uma melhor relação de proporcionalidade entre os valores das diferentes features, foi aplicado ao banco de dados esse modelo de aprendizagem de máquina, tendo como resultado 41% de acurácia

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns

# Dividindo dados para treino e dados para teste
x_entrada1 = x_inter3[['tumor','nodulo',
                       'metastese', 'tempo_de_amamentacao',
                       'primeira_menstruacao', 'fumante', 'alcolatra',
                       'grau_parentesco_1', 'grau_parentesco_2', 'grau_parentesco_3',
                       'tipo_tumor', 'idade']].values
y_saida1 = x_inter3['sobrevida'].values

X_train1, X_test1, Y_train1, Y_test1 = train_test_split(x_entrada1, y_saida1, 
                                                    test_size = 0.3, 
                                                    random_state = 42)
# Treinando o modelo
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X_train1, Y_train1)
# Fazendo as predições
Y_pred1 = model.predict(X_test1)

In [ ]:
Y_pred1
#no linear consigo pegar o score e accuracy

array([1.35471276, 1.83557527, 1.30048918, 1.71097868, 1.27503977,
       1.45773327, 2.00962601, 1.38638075, 1.60459134, 1.17885719,
       1.9587969 , 1.05324887, 1.99700382, 1.9600094 , 1.49653205,
       1.98286967, 1.21527868, 1.76511221, 1.92793119, 1.29124384,
       1.77407345, 1.9248693 , 3.20248877, 1.26717171, 1.1268893 ,
       1.79189427, 1.91973652, 1.91700152, 1.91857015, 1.24951764,
       0.89315453, 1.74489807, 1.42080076, 1.31863028, 1.55453784,
       1.49269412, 1.2341365 , 1.93442284, 1.26356403, 1.48591264,
       1.4181536 , 1.33364483, 1.45229729, 1.92211881, 1.34465205,
       1.95010892, 1.99687534, 1.46690216, 1.10028847, 1.35251293,
       1.95371895, 1.4189933 , 1.26272562, 1.31825446, 1.97751518,
       1.18099935, 1.18780358, 1.92036623, 1.56757578, 2.02615454,
       0.87473532, 1.91890649, 1.74270122, 1.36216652, 1.32910728,
       1.31832365, 1.78850353, 1.24638853, 1.96437247, 1.46521949,
       1.98056092, 1.21343111, 1.91616892, 1.20033549, 1.92802

In [ ]:
Y_test1

array([2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 3, 2, 2, 3, 3, 1, 2, 1, 2, 1,
       3, 2, 2, 1, 2, 3, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 2, 3, 3, 1, 2, 3, 1, 1, 1, 3, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2,
       3, 1, 4, 2, 5, 2, 1, 1, 3, 3, 1, 2, 1, 1, 1, 1, 3, 3, 1, 1, 1, 2,
       1, 3, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2,
       1, 2, 2, 1, 2, 3, 2, 4, 2, 1, 1, 2, 1, 1, 2, 3, 1, 2, 1, 1, 1, 1,
       3, 1, 4, 2, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 4, 1, 1, 1, 1, 2, 1, 2,
       3, 3, 3, 2, 1, 1, 2, 2, 2, 3, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 2, 2, 1, 2, 3, 2, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 2, 3, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 3, 2, 1, 1, 3, 1, 1, 2, 3, 1,
       1, 2, 1, 3, 1, 4, 2, 3, 2, 2, 1, 2, 2, 2, 1, 1, 3, 2, 1, 1, 1, 5,
       1, 3, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 3, 3, 2, 1, 2, 1, 1])

#**5.2 - Árvore de decisão**
Uma árvore de decisão é um mapa dos possíveis resultados de uma série de escolhas relacionadas. Ela permite permite prever categorias discretas com base na comparção de elementos pré-selecionados nesse modelo( no caso as features de teste) e retorna uma classificação para essa predição. Foi aplicado esse modelo de teste ao projeto, que teve como retorno uma acurácia de 42%.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
# Dividindo x e y

# Dividindo dados para treino e dados para teste
x_entrada2 = x_inter3[['tumor','nodulo','metastese', 'tempo_de_amamentacao', 'primeira_menstruacao',
                       'fumante', 'alcolatra', 'grau_parentesco_1', 'grau_parentesco_2', 'grau_parentesco_3',
                       'tipo_tumor', 'idade','benigno/maligno']].values
y_saida2 = x_inter3['sobrevida'].values

X_train, X_test, Y_train, Y_test = train_test_split(x_entrada2, y_saida2, 
                                                    test_size = 0.3, 
                                                    random_state = 42)
# Treinando o modelo
model = DecisionTreeClassifier(criterion='entropy', random_state = 42)

model.fit(X_train,Y_train)
# Fazendo as predições
Y_pred = model.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score
# accuracy_score(Y_test, Y_pred)

accuracy_score(Y_test, Y_pred, normalize=True)

0.421455938697318

#**5.3 - KNN**

A ideia principal do KNN é determinar o rótulo de classificação de uma amostra baseado nas amostras vizinhas advindas de um conjunto de treinamento. Dessa forma, é possível predizer um resultado com base na comparação dos dados mais próximos e retorná-lo em forma de uma classificação. Esse modelo de teste foi aplicado ao projeto, e teve como resultado uma acurácia de 57%.

In [ ]:
#Convertendo os dados da tabela em 0 e 1 para melhor se ajustar nesse modelo de predição 
valor = tabela_final.follow_up_days.median()
tabela_final['tempo_sobrevida'] = ["1 "if dias >= valor else "0"for dias in tabela_final["follow_up_days"]]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score, f1_score

knn = KNeighborsClassifier(n_neighbors=23)
knn.fit( X_train, Y_train)

print('Acc treino: ', knn.score(X_train, Y_train ))
print('Acc teste: ', knn.score(X_test, Y_test.squeeze() ))

Acc treino:  0.5763546798029556
Acc teste:  0.47509578544061304


#6.0 - Análise dos resultados

Em um primeiro momento, foi decidido como parâmetro de avaliação de desempenho a acurácia dos modelos de aprendizagem de máquina, visto que essa métrica engloba todas possíveis variáveis de resultado, como os verdadeiros positivos, verdadeiros negativos, falsos positivos e falsos negativos.

Dessa forma, com base nos resultados finais obtidos nos testes, conclui-se que o melhor método de aprendizagem de máquina para este projeto até o momento é o KNN, com uma acurácia de 57%.

Esse valor indica que , dentre todos os resultados dos testes de predições, o número de valores verdadeiros positivos e negativos representam 57% do total, o que sugere uma revisão nos tratamentos dos dados de modo a eliminar os falsos positivos e negativos, responsáveis por abaixar essa porcentagem e diminuir a precisão e recall do modelo, o que consequentemente abaixam a acurácia de modo geral.

Além disso, acredita-se que esse resultado possa ser ainda otimizado baseado em outros modelos de classificação mais sofisticados, como o svm, que deverá ser implementado futuramente visando buscar melhores resultados e métricas de comparação.

